# Imports

In [11]:
%%time
import twitter
import csv
import re
import nltk
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords
from textblob import TextBlob 
import numpy as np
nltk.download('stopwords')
nltk.download('punkt')

Wall time: 1.99 ms


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bless\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bless\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
%time
twitter_api = twitter.Api(consumer_key ='8i68MvUnFgp3Mqh5jRV34jGh3',
                        consumer_secret= 'jNPsW7ZKZkKoVLAvFjwgTPHvlw4Hto8jbpnCDcbnnSUm7O8RJX',
                        access_token_key='2378518193-5lkOIoa6OJpm3X0AehYz9EAjws6NjSsjHZS93MM',
                        access_token_secret = 'DN6j3LxuKWtkKJkGpU16pUr2wqSFYoUs6FwED1SXFjpAw')

In [13]:
#Building the test data set
def buildTestSet(search_keyword):
    try:
        tweets_fetched = twitter_api.GetSearch(search_keyword, count = 100, lang = "en")
        
        print("Fetched " + str(len(tweets_fetched)) + " tweets for the term " + search_keyword)
        
        return [{"text":status.text, "label":None} for status in tweets_fetched]
    except:
        return None
    pass

In [14]:
%%time
#Building the training data set
"""This is done in a seperate file"""
#import TrainingTweetsBuilder

'This is done in a seperate file'

In [16]:
def AccessCSV(train_data):
    trainDataSet = [] 
    with open(train_data,'r', encoding = "utf-8") as csvfile:
        lineReader = csv.reader(csvfile,delimiter=',', quotechar="\"")
        for row in lineReader:
                trainDataSet.append(row)
    return trainDataSet
  

In [17]:
#Create and implement PreProcessor
class PreProcessTweets:
    
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            if any(isinstance(tweet,dict) for tweet in list_of_tweets):
                try:
                    processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
                except Exception as e:
                    print(e)
            else:
                try:
                    processedTweets.append((self._processTweet(tweet[1]),tweet[2]))
                except Exception as e:
                    print(e)
                
        return processedTweets
    
    def _processTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet =  re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet =  word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        
        return [word for word in tweet if word not in self._stopwords]

In [18]:
#Prepping the model
def buildVocabulary(data_to_train):
    all_words = []
    
    for (words, sentiment) in data_to_train:
        all_words.extend(words)

    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    return word_features
def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in tweet_words)
    return features 

## The whole program

In [19]:
%%time

#Build test set
search_term = input("Search on Twitter:")
testDataSet = buildTestSet(search_term)
print(testDataSet[:5])

Search on Twitter:George Floyd
Fetched 100 tweets for the term George Floyd
[{'text': 'For George Floyd.\nFor Breonna Taylor.\nFor our children.\nFor \u200ball\u200b of us.\n\nAnd for the lives of too many others to n… https://t.co/76y6vV6ADw', 'label': None}, {'text': 'Omg Kamala just said with a straight face “The Coronavirus is racist and knows how we see and treat each other” and… https://t.co/ACnKmxiGPV', 'label': None}, {'text': 'The freak who brutally attacked an innocent passerby in Portland is not just an activist — he’s a self-identified B… https://t.co/cITjdC52wl', 'label': None}, {'text': 'Office Williams Justice for George Floyd https://t.co/WBkBnqUu7u via @YouTube', 'label': None}, {'text': 'RT @bluelivesmtr: Attorney Files To Dismiss Charges In George Floyd Case Saying He Died Of Overdose - The Police Tribune https://t.co/0tqi5…', 'label': None}]
Wall time: 5.79 s


In [20]:
%%time
#Access saved training set
train_data =  "tweetDataFile.csv"     
trainDataSet = list(filter(None, AccessCSV(train_data)))

Wall time: 20 ms


In [21]:
%%time

#Clean the training and test data, prepare features (words in the tweets) and target (sentiment)
tweetProcessor = PreProcessTweets()
data_to_train = tweetProcessor.processTweets(trainDataSet)
data_to_test = tweetProcessor.processTweets(testDataSet)
word_features = buildVocabulary(data_to_train)
trainingFeatures = nltk.classify.apply_features(extract_features, data_to_train)

Wall time: 3.84 s


In [24]:
%%time
#Train the learning model (in this case, a Naive Bayes Classifier) and apply testing tweets to test tweets 
NBayesClassifier = nltk.NaiveBayesClassifier.train(trainingFeatures)
NBResultLabels = [NBayesClassifier.classify(extract_features(tweet[0])) for tweet in data_to_test]

Wall time: 1min 46s


## Error Analysis

In [25]:
%%time
X = 0
Y = 0 
Z = 0 
testDataSet_copy = testDataSet.copy()
for tweet in testDataSet_copy:
    if  TextBlob(tweet['text']).sentiment.polarity > 0:
        tweet['label'] = "positive"
        X+= 1
    elif TextBlob(tweet['text']).sentiment.polarity < 0:
        tweet['label'] = "negative"
        Y += 1
    else:
        tweet['label'] = "neutral"
        Z += 1

Wall time: 323 ms


In [26]:
sentiment = []
for tweet in testDataSet_copy:
    sentiment.append(TextBlob(tweet['text']).sentiment.polarity)
real_sentiment = np.mean(sentiment)

In [27]:
matches = 0
total = len(testDataSet)
for i, label in enumerate(NBResultLabels):
    if label == testDataSet_copy[i]['label']:
        matches += 1
accuracy = 100*(matches/total)



In [28]:
if NBResultLabels.count('positive') > NBResultLabels.count('negative'):
    print("Predicted: Overall Positive Sentiment")
    print(f"Positive Sentiment Percentage = {100*NBResultLabels.count('positive')/len(NBResultLabels)}%")
else: 
    print("Predicted: Overall Negative Sentiment")
    print(f"Negative Sentiment Percentage = {100*NBResultLabels.count('negative')/len(NBResultLabels)}%")

print ("  ")

#get the real sentiment
if real_sentiment > 0:
    print ('Actaul: Overall Positive sentiment')
    print(f"Positive Sentiment Percentage = {100*X/total}%")
    
elif real_sentiment < 0:
    print ("Actual: Overall Negative Sentiment")
    print(f"Negative Sentiment Percentage = {100*Y/total}%")

else:
    print (f"Actual: Overal Neutral Sentment with a percentage of {100*Z/total}%'")
    
print("  ")
print(f"The sentiment of {matches} out of {total} tweets was predicted correctly")
print(f'Accuracy of prediction = {round(accuracy,2)}%')
    

Predicted: Overall Negative Sentiment
Negative Sentiment Percentage = 3.0%
  
Actaul: Overall Positive sentiment
Positive Sentiment Percentage = 42.0%
  
The sentiment of 46 out of 100 tweets was predicted correctly
Accuracy of prediction = 46.0%


In [ ]:
0